# <span style="color:#F4563B"> Prédiction du risque de défaut des clients d'une microfinance à but non lucratif : Cas de Kiva </span>

# <a name="cadregén"></a> <span style="color:#FA8072"> Cadre général de l'étude </span>

*En introduction de ce projet, présentons le contexte scolaire dans lequel il est rédigé à l'ENSAE, le contexte professionnel dans lequel il est étudié du point de vue de l'entreprise concernée, ainsi que la problématique traitée et le plan suivant lequel elle sera présentée.*

## <span style="color:#F5B7B1"> Contexte scolaire </span>

L'***ENSAE*** est l'une des principales Grandes Écoles françaises spécialisée en Économie, Data Science, Sciences Sociales Quantitatives, Finance et Actuariat. Parmi les enseignements fondamentaux de cette École, figurent les compétences informatiques pour lesquelles le cours ***Python pour le data scientist/ économiste*** est administré aux étudiants de deuxième année.

Il s'agit de les initier à la ***manipulation***, la ***visualisation*** et la ***modélisation de données*** en ***Python***, langage aujourd'hui très présent chez les scientifiques. Dans le but de confronter les enseignements reçus à des problématiques réelles, des thématiques et bases de données à analyser sous forme de projets ont été demandées aux étudiants répartis en groupes de deux (02) ou de trois (03).

C'est dans ce cadre que nous avons choisi, avec l'accord des enseignants, de travailler sur du ***credit scoring*** sur une base de la microfinance à but non lucratif ***Kiva*** collectée sur leur [site internet](https://www.kiva.org/build/data-snapshots) le ***20 Octobre 2020***.

## <a name="contexteprof"></a> <span style="color:#F5B7B1"> Contexte professionnel (fonctionnement de Kiva) </span>

***Kiva*** est une organisation internationale à but non lucratif, plateforme de [financement participatif](https://www.economie.gouv.fr/cedef/financement-participatif) fondée en 2005 à San Francisco en solution au problème de non bancarisation touchant encore 1,7 milliards de personnes dans le monde aujourd'hui (Source : [site de Kiva](https://www.kiva.org/about)). Elle met en relation des prêteurs (à capacité de financement) et des projets (à besoin de financement). A ce jour, 1,51 millards de prêts ont été financés par 2 921 prêteurs à travers Kiva (Source : [site de Kiva](https://www.kiva.org/about)).

Deux (02) types de prêteurs se distinguent : d'une part les particuliers, et d'autre part les entreprises, organisations ou fondations partenaires. Aucun d'entre eux ne reçoit d'intérêt, puisqu'aucun intérêt n'est demandé aux emprunteurs. Les prêteurs particuliers n'y gagnent donc rien financièrement, et les partenaires reçoivent parfois de petits frais de service de la part de Kiva. En effet, l'organisation dispose de fonds collectés chaque jour auprès de donateurs, qui leur permettent de rémunérer les 110 employés (à la date du 11/12/2020, source : [site de Kiva](https://www.kiva.org/about)) qui y travaillent, de financer les petits frais de service des partenaires, de prêter directement aux entrepreneurs et de couvrir tous les frais d'opérations liés à leur activité.

Par ailleurs, en situation d'incapacité de remboursement d'un prêt, aucun dédommagement n'est assuré par Kiva. Les fournisseurs doivent simplement considérer le solde impayé comme une perte. Pourtant, les institutions financières telles que nous les connaissons (banques, microfinances, etc...), tirent leurs bénéfices des intérêts que les emprunteurs payent au moment du remboursement. Si un prêt est remboursé après le délai, cela constitue un manque à gagner, et s’il n’est pas remboursé, cela constitue en plus une perte de capital lorsqu’aucune véritable garantie de paiement n’est exigée à l’emprunteur.

Ainsi, en tant que prêteur (intermédiaire des prêteurs), Kiva est nécessairement confrontée à la problématique de non remboursement des prêts. Toutefois, à la différence des banques et microfinances pour lesquelles le remboursement est une question, non seulement de pérennité mais surtout de rentabilité, Kiva s'y interesse pour limiter les pertes des particuliers prêteurs et de ses partenaires, et ainsi encourager les dons et partenariats pour assurer la perrenité de ce service dont l'utilité est évidente.

Consciente de cela, Kiva a mis en place un système de sélection et de suivi et évaluation des projets à financer. Pour être en accord avec cette idée de fournir du financement à ceux qui ne sont pas en mesure de les obtenir auprès des banques, Kiva a mis en place un système de garantie (orienté microfinance) plus adapté aux agents qui ne sont pas en mesure de disposer de garanties telles qu'exigées par les banques (propriétés ou autres capitaux disponibles), sur la base duquel il approuve ou pas le prêt : la **"[souscription sociale](https://www.kiva.org/businesscenter/what-is-social-underwriting)"**.

Il est question pour l'emprunteur de rechercher lui-même, durant une **"période de collecte privée de fonds"**, cinq (05) à ving-cinq (25) prêteurs (particuliers ou organisations) de sa communauté qui se porteront garants de lui en lui prêtant une certaine somme. Les emprunteurs de Kiva peuvent également (facultatif) trouver des administrateurs qui se portent garants de leurs prêts en mettant leur réputation et leur crédibilité en jeu, et/ ou fournir trois (03) références (clients, fournisseurs, petits commerces de sa communauté, etc...).

Le système de sélection consiste alors en un jugement de la réputation d'un emprunteur et de la crédibilité de son projet ; le premier se déduisant simplement de la réputation de ses références, administrateurs ou garants, et le second relevant de l'étude de la faisabilité et de la rentabilité du projet pour lequel le financement est demandé. Après financement, le projet est suivi et évalué par les experts de Kiva pour s'assurer de leur bonne gestion. Et, si l'emprunteur se retrouve tout de même dans la difficulté de rembourser son prêt, le partenaire ou Kiva (le prêteur) rééchelonne le prêt afin de lui faciliter le remboursement.

## <span style="color:#F5B7B1"> Problématique et objectifs du travail </span>

Dans le même sillage que le processus de sélection assez subjectif présenté, ce travail se propose de répondre d'une façon plus objective à la question : à qui prêter ? Cette problématique relève du credit scoring et cette étude est conduite, bien qu'en premier lieu dans une optique de mise en pratique de la programmation en language Python, également dans le but d’élaborer un modèle statistique qui puisse permettre de séparer les emprunteurs (projets) potentiels de Kiva entre ceux pour qui des fonds pourraient être collectés et ceux à qui on refuse le financement.

Concernant l'aspect "programmation", les techniques suivantes (liste non exhaustive) seront appliquées :
* importation dynamique et statistique de la base de données ;
* webscrapping ;
* représentation spaciale ;
* natural language processing (NLP) ;
* etc...

Par alleurs, quant à la modélisation, il sera spécifiquement question :
* d’analyser les caractéristiques individuelles des emprunteurs (projets) liées à l'échec ;
* de dégager le profil des projets qui sont remboursés et le profil de ceux qui ne le sont pas ;
* de construire un modèle qui rend compte, sous la forme d’un score, du niveau de risque que représente chaque projet ;
* de proposer une échelle de notation des projets suivant leurs scores ;
* de proposer une stratégie permettant d’évaluer les projets à l'aide du modèle construit.

## <span style="color:#F5B7B1"> Hypothèse </span>

Le travail qui sera effectué repose sur l’hypothèse principale qu’il existe une différence dans les caractéristiques des projets qui sont remboursés et de ceux qui ne le sont pas.

## <span style="color:#F5B7B1"> Plan du travail </span>

<span style="color:#FA8072"> Section I) Cadre théorique de l'étude </span>

<span style="color:#F5B7B1"> I) Généralités sur la microfinance à but non lucratif et le credit scoring </span>

<span style="color:#F5B7B1"> II) Revue de littérature et méthodologie de l'étude </span>

<span style="color:#FA8072"> Section II) Cadre pratique de l'étude </span>

<span style="color:#F5B7B1"> I) Preprocessing et visualisation uni-variée </span>

<span style="color:#F5B7B1"> II) Analyse multi-variée et visualisation multi-variée </span>

<span style="color:#F5B7B1"> III) Modélisation </span>

# <span style="color:#FA8072"> Cadre théorique de l'étude </span>

*Avant de se lancer dans les analyses proprement dites, il convient de présenter le concept de microfinance à but non lucratif qui pourrait être nouveau pour beaucoup, d'expliquer succintement le processus d'emprunt chez Kiva, de récapituler quelques travaux sur le credit scoring ayant inspiré la méthodologie que nous avons adopté et de dérouler cette méthodologie.*





## <span style="color:#F5B7B1"> Généralités sur la microfinance à but non lucratif et le credit scoring </span>

*Nous débutons cette section par la présentation du concept de microfinance à but non lucratif.*

### <span style="color:#FED2CA"> La microfinance à but non lucratif </span>

*Tout d'abord, la microfinance, qu'est-ce que c'est ?*

#### La microfinance

La microfinance se définit comme un ensemble de services financiers (crédit, assurance, épargne) destinés aux pauvres exclus du système bancaire traditionnel pour leur permettre de financer une activité génératrice de revenu, pour leur inclusion financière. L'on assimile très souvent la microfinance à son produit le plus populaire, le ***microcrédit***. Etant donné que notre projet ne concerne que ce dernier produit, nous vous proposons de nous arrêter quelques instants sur ce concept.

##### Etymologie du microcrédit

À ce jour, il n’existe pas de définition universelle du microcrédit. Cependant, l’idée de base retrouvée dans toutes les définitions est contenue dans l’étymologie de ce mot. Pour comprendre le concept de microcrédit, il est alors sagace de revenir à sa genèse et à son histoire.

Le mot *« microcrédit »* est composé à partir de l’adjectif grec *« mikros »*, généralement traduit en *« petit »* ou *« un peu »*, qui qualifie tout ce qui est de faible dimension ; et du verbe latin *« credere »* qui se traduit en *« faire confiance à »*, *« faire confiance pour »* et également *« confier en prêt »*. Au sens brut, un microcrédit renvoie donc simplement à un prêt de faible dimension, la dimension étant ici le montant. Mais la conception du microcrédit ne se limite pas à cette idée, et provient du contexte dans lequel cette notion est née.

##### Origine du microcrédit

Bien que des antécédents puissent être retracés dans plusieurs endroits dans le monde, le microcrédit doit sa popularité à l’initiative du ***Dr Muhammad Yunus*** au début des années 70. Avec ses étudiants de l’université de Chittagong au Bangladesh, le professeur d’économie constate que de nombreux marchands et artisans ont besoin de petites sommes d’argent pour développer leurs activités mais ne peuvent pas les obtenir auprès des banques car, pauvres, ils ne possèdent pas de garantie et les montants sollicités sont trop faibles pour générer les bénéfices souhaités.

Il s’engage alors à prêter de l’argent à ces artisans sous des conditions plus raisonnables : un prêt n’est accordé qu’à un groupe d’artisans qui pourront développer leurs activités séparément mais qui devront le rembourser ensemble. En d’autres termes, chacun des artisans est garant des autres.

Ainsi, il conçoit un nouveau système de garantie en marge de celui pratiqué par les banques, basé sur la confiance et la solidarité. Satisfait de l’utilité procurée par le premier prêt, Muhammad Yunus crée en 1976 au Bangladesh la Grameen Bank, première institution spécialisée dans le microcrédit et première tentative de solution entrepreneuriale formelle moderne à l’exclusion bancaire des plus pauvres.

Il doit à cette initiative le surnom « banquier des pauvres » et le prix Nobel de la paix reçu en 2006. Avec le succès de ce nouveau système de prêts perceptible à l’évolution des activités locales qui témoigne de son utilité et à son taux de remboursement de l’ordre de 97% pour des taux d’intérêts élevés qui démontre sa rentabilité, ce modèle ne tarda pas à être copié un peu partout dans le monde, pour aider les plus pauvres à démarrer des activités génératrices de revenus.

##### Définition du microcrédit

C’est ainsi que le microcrédit s’associe, non seulement à un prêt de faible montant, mais également à un prêt destiné aux pauvres ne disposant ni d’apport personnel suffisant, ni de la garantie exigée par les banques traditionnelles. Sans garantie et les montants demandés étant trop faibles donc jugés peu rentables, ces banques ne sont pas intéressées à accorder de tels prêts.

#### La microfinance à but non lucratif

Le microcrédit correspond alors au financement offert aux populations exclues du système bancaire classique ([Ellé, 2017](https://doi.org/10.1016/j.rgo.2017.09.004)). En général, les microcrédits offrent aux pauvres la possibilité d’emprunter de l’argent contre un accord pour rembourser ces fonds avec intérêts et/ou frais à une/plusieurs date(s) future(s) fixée(s). Mais ce modèle, qualifié de ***microfinance à but lucratif***, bien qu'étant le plus répandu, n'est pas le seul appliqué. Kiva, Babyloan, des organisations ***à but non lucratif*** proposent à ces personnes des prêts ***sans intérêt*** et ***sans frais***. Leur activité est qualifiée de ***microfinance à but non lucratif***.

*Nous rebondissons directement sur l'activité de Kiva dont le fonctionnement général a été présenté dans le [contexte professionnel](#contexteprof) de ce projet. Il ne reste qu'à dérouler le processus d'emprunt.*

#### Processus d'emprunt chez Kiva

Les étapes d'un prêt chez Kiva sont les suivantes :

* **La demande de prêt** : L'emprunteur adresse une demande de prêt soit directement à Kiva, soit à un partenaire.
* **L'étude de la demande de prêt** : Si la demande est directement adressée à Kiva, c'est le système de [souscription sociale](#contexteprof) qui permet à l'organisation de décider si elle approuve le prêt ou pas. Autrement dit, Kiva approuve ou désapprouve le prêt selon la réputation et la crédibilité des références de l'emprunteur. Si la demande est plutôt adressée à un partenaire, celui-ci se charge d'approuver ou de refuser avec l'aide d'institutions de crédit ou d'autres organisations à but non lucratif. Kiva procède toutefois au suivi du prêt pour le partenaire.
* **La collecte privée de fonds** : Comme expliqué dans le principe de [souscription sociale](#contexteprof), l'emprunteur dispose d'une certaine période pour obtenir du financement auprès de prêteurs de son entourage.
* **La collecte publique de fonds** : Le prêt est posté sur le site de Kiva. L'emprunteur doit activement faire campagne pour recueillir le plus de fonds possible et un financement entier dans un délai de trente (30) jours pour pouvoir recevoir l'argent. Les prêteurs peuvent contribuer à partir de 25 dollars.
* **Le remboursement** : Les prêteurs sont remboursés au fur et à mesure, suivant le calendrier initialement fixé. Ils sont directement remboursés sur leurs comptes Kiva et peuvent retirer, faire des dons ou financer d'autres prêts avec ces fonds.

*Ayant terminé avec la présentation de Kiva et de la microfinance à but non lucratif, nous poursuivons avec le second concept clé de ce travail : le ***credit scoring***.*

### <span style="color:#FED2CA"> Présentation du credit scoring </span>

*Dans les établissements de crédit, le recours au credit scoring est une pratique très populaire permettant d’évaluer, de mesurer, de quantifier le risque de crédit. Ainsi, avant de passer au credit scoring proprement dit, la notion de risque de crédit sera abordée.*

#### Le risque de crédit

##### Définition

Lorsqu’un agent (individu, ménage, entreprise, établissement de crédit) octroie un prêt à un autre agent, il existe toujours pour le prêteur un risque appelé risque de crédit. C'est un concept très important et très courant dans les établissements financiers, pour lequel de nombreux auteurs ont proposé des définitions pouvant être synthétisées comme suit :

Le *risque de crédit* est le risque de non-remboursement de ses dettes par un emprunteur défaillant. Il s’assimile au degré d’incertitude qui pèse sur l’aptitude d’un emprunteur à rembourser sa dette, c’est-à-dire à l’incertitude des pertes pouvant être générées par un crédit donné.

##### Composantes

Le risque de crédit se décompose en trois éléments :

- le **risque de défaut** qui est la probabilité d’un manquement ou d’un retard de l’emprunteur sur le paiement de sa dette (aux échéances fixées) ;
- le **risque de recouvrement en cas de défaut** ou risque lié à l’incertitude du recouvrement une fois le défaut survenu ;
- et le **risque de dégradation de la qualité de crédit** qui est le risque que la qualité perçue de l’emprunteur se dégrade pendant la durée du crédit.

*Dans le cadre de ce projet, nous nous intéressons uniquement au risque de défaut des emprunteurs de Kiva.*

##### Risque de défaut

La ***défaillance*** c’est la situation dans laquelle l’emprunteur n’est plus en mesure d’honorer ses engagements financiers. Dans la pratique, chaque prêteur a la latitude de définir le défaut, c’est-à-dire les conditions dans laquelle un emprunteur est considéré comme défaillant.

En Italie par exemple, l’association des banques indique que le défaut peut correspondre à un arriéré de 180 jours tandis qu’en Espagne, un arriéré de 90 jours semble acceptable (Gouriéroux & Tiomo, 2007). La situation de défaillance (défaut) est donc définie par chaque pays et par chaque établissement de crédit selon ses spécificités.

*Après ce petit tour de la notion de risque de crédit appréhendé par le credit scoring, nous passons à la présentation du credit scoring proprement dit.*

#### Le credit scoring

##### Définition

Le credit scoring est une méthode très utilisée dans les établissements de crédit, intégrée dans la décision d’octroi ou de rejet de crédit. Plusieurs auteurs ont tenté de définir ce concept. Quelques unes des définitions les plus partagées dans la littérature sont ci-après citées.

- « Le credit scoring est une méthode d’évaluation du risque de crédit des demandes de prêt. À partir de données historiques et de techniques statistiques, le credit scoring tente d’isoler les effets de plusieurs caractéristiques des emprunteurs sur le défaut de paiement. La méthode produit un « score » qu’une banque peut utiliser pour classer ses potentiels emprunteurs en termes de risque. » ([Mester, 1997](https://www.researchgate.net/profile/Loretta_Mester/publication/5051659_What_Is_the_Point_of_Credit_Scoring/links/54b668cf0cf2bd04be32098a.pdf))
- « Le credit scoring est le recours aux modèles statistiques en vue de transformer des données pertinentes en des indicateurs numériques mesurables à des fins d’aide à la décision d’octroi ou de refus de crédit. » ([Anderson, 2007](https://global.oup.com/academic/product/the-credit-scoring-toolkit-9780199226405?cc=us&lang=en&#))

De ces différentes définitions, il ressort que le credit scoring prend en entrée un historique d’un ensemble de caractéristiques liées à des prêts antérieurement contractés, construit à l’aide de techniques statistiques un modèle qui prédit, sous la forme d’une note (ou « score »), la probabilité qu’un potentiel emprunteur ne rembourse pas son prêt comme prévu (défaut) et qui guide le prêteur pour décider s’il lui octroie ou pas un crédit.

*Pour résoudre les problématiques de credit scoring, la littérature nous oriente vers un schéma assez précis ci-après déroulé.*

##### <a name="schemares"></a> Schéma de résolution d’une problématique de credit scoring

Disposant de l’historique de caractéristiques liées à des prêts antérieurement contractés nécessaire, la construction de modèles de credit scoring implique généralement le processus suivant, présenté par [N. Chen et al. (2016)](https://doi.org/10.1007/s10462-015-9434-x) :

* **Définition du défaut de remboursement** : elle s’assimile à la définition de la variable cible, une variable qualitative à deux classes indiquant si un emprunteur/ projet est « bon » ou « mauvais », c’est-à-dire s’il est digne de confiance/ prometteur ou pas. Cette qualité d’être « bon » ou « mauvais » est exprimée en fonction du défaut de remboursement. Par exemple, un-emprunteur peut être jugé mauvais s’il enregistre au moins un défaut de remboursement sur une période donnée. Préciser le sens considéré du défaut de remboursement est donc primordial.
* **Pré-traitement des données** : il s’agit du traitement des valeurs manquantes, de la normalisation, du filtrage, etc. . . appliqués aux données constituant l’historique de caractéristiques liées aux prêts.
* **Sélection des variables et réduction de la dimensionnalité** : cela consiste au filtrage des caractéristiques (variables) et emprunteurs (individus) non pertinents. Les outils de sélection des variables (Analyse en Composantes Principales ou ACP, Analyse des Correspondances Multiples ou ACM, analyse discriminante, etc. . . ) servent à la réduction de la dimensionnalité de l’ensemble de données.
* **Implémentation des modèles** : différents algorithmes sont implémentés à partir des données issues de la phase précédente pour prédire la variable cible à deux modalités.
* **Évaluation du modèle** : La performance des prédictions est évaluée sur un ensemble test pour comparer les différents algorithmes implémentés.

In fine, on obtient un modèle prédisant la probabilité de défaut de remboursement d’un potentiel emprunteur à partir de ses caractéristiques, probabilité qui pourra être convertie sous forme de score puis associée à une échelle de risque qui, elle-même, ne pourra qu’orienter le décideur (ici Kiva) dans sa décision d’octroi ou de rejet de crédit puisque ce dernier devra décider à partir de quel seuil les demandes seront rejetées, acceptées ou encore mieux étudiées.

*Le credit scoring a à son compte de nombreux avantages, et présente également de nombreuses limites.*

##### Quelques avantages et limites du credit scoring

Parmi les avantages du credit scoring, on cite :

* l’amélioration de l’efficience dans le processus d’analyse et de livraison du crédit (Kulkosky, 1996) : le calcul du score de crédit est rapide, ce qui non seulement permet une prise de décision également rapide, mais autorise l’automatisation du processus d’octroi de crédits ;
* la réduction de la discrimination entre les clients : le credit scoring fournit une évaluation objective de la solvabilité des emprunteurs ; deux clients ayant les mêmes caractéristiques sont traités de la même façon par le modèle de credit scoring contrairement à l’évaluation subjective d’agents dans laquelle les affinités et sentiments sont prises en compte ;
* la souplesse de la méthode, permettant de modifier la politique de crédit simplement en jouant sur le seuil de sélection des dossiers.

Entre autres limites du credit scoring, on peut citer :

* l’hypothèse fondamentale de toute modélisation prédictive est que le passé peut prédire l’avenir ([Berry & Linoff, 2000](https://doi.org/10.1108/imds.2000.100.5.245.2)), pourtant l’avenir peut être différent du passé : les prédictions de défaut du credit scoring s’appuient uniquement sur les comportements passés et parfois, ceux-ci changent rapidement au cours du temps nécessitant une mise à jour constante des modèles ;
* l’omission de variables importantes ([Avery et al., 2000](https://www.federalreserve.gov/pubs/bulletin/1996/796lead.pdf)) : le défaut de crédit peut également s’expliquer par des variables qui ne sont pas prises en compte dans le modèle ;
* le modèle peut être construit en utilisant un échantillon biaisé de clients à qui des crédits ont été accordés ([Hand, 2001](https://doi.org/10.1093/imaman/12.2.139)) : ceci s’explique principalement par le fait que les clients à qui des crédits ont été refusés ne soient pas inclus dans l’échantillon de base du modèle puisqu’il n’est pas possible de se prononcer sur leur défaillance ; une attention particulière doit ainsi être prêtée pour limiter au mieux le biais causé par l’écart entre le profil des clients composant l’échantillon d’origine et celui de l’ensemble des clients.

*Tout au long de ce travail, des techniques de data mining seront utilisées. Nous choisissons de justifier cela en présentant l'utilité que peut avoir le data mining dans la résolution d'une problématique de credit scoring.*

##### Credit scoring et data mining

Une définition pratique du data mining est « la découverte de structures intéressantes, inattendues, ou de grande valeur dans un vaste ensemble de données » ([Hand et al., 2001](https://doc.lagout.org/Others/Data%20Mining/Principles%20of%20Data%20Mining%20%5BHand%2C%20Mannila%20%26%20Smyth%202001-08-01%5D.pdf)). Ces dernières années, les techniques de data mining ont été de plus en plus utilisées pour construire des modèles de credit scoring ([Peng & Chwee, 2004](https://go.gale.com/ps/anonymous?id=GALE%7CA119370565&sid=googleScholar&v=2.1&it=r&linkaccess=abs&issn=01295977&p=AONE&sw=w)).

En effet, l’élaboration de modèles de credit scoring nécessite l’exploitation de grandes masses de données, matière dans laquelle excelle le data mining. À cet effet, les trois grandes catégories de techniques de data mining sont généralement nécessaires : les techniques de description et de visualisation, les techniques d’association et de segmentation, et les techniques de classification et d’estimation (ou de régression).

* La **description** et la **visualisation** interviennent au commencement du processus, pour identifier les structures très souvent complexes dissimulées dans ces énormes masses de données. Les techniques de description et de visualisation (Analyse en Composantes Principales ou ACP, l’Analyse des Correspondances Multiples ou ACM, l’Analyse Factorielle Discriminante ou AFD) permettent ainsi de comprendre et d’appréhender les ensembles de données du credit scoring.

* Les techniques d’**association** (algorithmes à l’instar de l’algorithme Apriori, l’algorithme OPUS) et de **segmentation** (algorithmes k-moyennes (k-means), espérance-maximisation (expectation-maximization algorithm), ...) servent respectivement à déterminer quelles variables s’associent (on parle d’extraire des règles d’association), et à regrouper les objets similaires en séparant ceux qui ne le sont pas. Elles interviennent pour identifier les liens et associations existants entre les caractéristiques sélectionnées pour la construction des modèles de credit scoring.

* La **classification** se définit comme la prédiction d’une variable cible catégorielle tandis que l’estimation et la **régression** fait référence à la prédiction d’une variable cible quantitative. En credit scoring, il s’agit généralement d’une variable à deux modalités indiquant si un client a fait défaut ou pas. Elle peut être directement prédite à l’aide de techniques de classification telles que les arbres de décision, ou encore sous forme de probabilité que le client prenne une des modalités à l’aide de techniques de régression telles que la régression logistique.

Ainsi, le recours au data mining dans le processus de construction de modèles de credit scoring est une pratique justifiée et appropriée, et également courante dans la littérature.

*Ayant acquis les notions essentielles sur le fonctionnement de Kiva et le credit scoring, nous passons à présent à la présentation de quelques travaux empiriques portant sur la question étudiée et au déroulement de la méthodologie qui sera mise en oeuvre pour la résoudre.*






## <span style="color:#F5B7B1"> Revue de la littérature et méthodologie </span>

### <span style="color:#FED2CA"> Revue de la littérature </span>

La problématique du défaut de remboursement est une problématique très abordée dans la littérature, beaucoup plus traitée dans les banques mais de plus en plus étudiée dans les microfinances à but non lucratif également. De nombreux articles et ouvrages décrivant chacun une problématique de credit scoring bien précise, les méthodes utilisées pour la modélisation et les résultats obtenus existent, et également ceux étudiant les facteurs explicatifs du défaut de remboursement existent. Pour notre étude, nous nous sommes inspirés de plusieurs d'entre eux, dont un en particulier porte directement sur les données étudiées.

Il s'agit de la thèse intitulée **"[Predicting a successful loan in prosocial lending: the role of partner accreditation and storytelling on Kiva.org](https://www.researchgate.net/publication/326426143)"** rédigée par Martina Pocchiari. Le principal apport de ce document à notre travail concerne l'appréhension du défaut de remboursement, élément clé de notre projet comme présenté dans le [schéma de résolution d'une problématique de credit scoring](#schemares). La définition du défaut de remboursement (donc de la variable cible) retenue sera alors précisée lors de la présentation de la base de données de l'étude.

Désirant que ce projet reste principalement axé sur l'application des techniques de Python, seuls les principaux enseignements tirés des différents travaux parcourus ([M. Godquin (2004) : Microfinance Repayment Performance in Bangladesh: How to Improve the Allocation of Loans by MFIs](https://www.researchgate.net/publication/222651429_Microfinance_Repayment_Performance_in_Bangladesh_How_to_Improve_the_Allocation_of_Loans_by_MFIs), [S. Kumar (2007) : Bank of One: Empirical Analysis of Peer-to-Peer Financial Marketplaces](https://www.semanticscholar.org/paper/Bank-of-One%3A-Empirical-Analysis-of-Peer-to-Peer-Kumar/0c9ef0db8b24aadebc928a806e0af0b44a3d9e91), [J. Ledgerwood (1998) : Microfinance Handbook: An Institutional and Financial Perspective](http://hdl.handle.net/10986/12383), [J. Galak, D. Small, A. T. Stephen (2011) : Microfinance Decision Making: A Field Study of Prosocial Lending](https://doi.org/10.1509/jmkr.48.SPL.S130), etc...) seront déroulés ici. Ainsi, ces travaux nous ont enseigné d'une part, que les possibilités sont très ouvertes en ce qui concerne le choix des variables à inclure dans les modèles de credit scoring et, d'autre part, qu'il n'existe pas de technique de modélisation unanimement meilleure que les autres. Toutefois, les techniques que nous avons recensées parmi celles ayant montré les meilleurs performances sont les arbres de décision, la régression logistique et les forêts aléatoires.


### <span style="color:#FED2CA"> Méthodologie de l'étude </span>

*Cette sous-section traite de la méthodologie choisie pour résoudre la problématique de credit scoring posée par l’étude. Elle expose les différentes étapes qui seront réalisées après l'importation dynamique et statique.*

Pour atteindre l’objectif d’évaluation du niveau de confiance à accorder aux emprunteurs et leurs projets, il sera question de construire un modèle prédisant la probabilité que ces derniers fassent défaut à partir de leurs caractéristiques, de convertir cette probabilité sous forme de score puis d’associer à ces scores une échelle de risque qui orientera Kiva dans le choix des projets à approuver. Ceci nécessitera l’exploitation d’un grand ensemble de données. Ceci étant, pour la réalisation de projets d’analyse de grands volumes de données comme c’est le cas de ce travail, il existe un processus standard très utilisé développé par IBM dans les années soixante : il s’agit de la méthode CRISP-DM (Cross Industry Standard Process for Data Mining). Cette étude sera réalisée suivant cette méthodologie.

La méthode CRISP-DM (Cross Industry Standard Process for Data Mining) consiste en six (06) phases, chaque phase étant constituée de diverses tâches à réaliser. Ces six phases sont : la compréhension du problème métier, la compréhension des données, la préparation des données, la modélisation, l’évaluation et le déploiement (Larose, 2005). Il n’existe pas d’ordre préétabli pour l’exécution des différentes phases. Selon le résultat escompté, la méthode peut être personnalisée en réalisant des phases avant d’autres, en faisant des allers-retours entre des phases ou encore en priorisant des phases par rapport à d’autres. Le schéma retenu pour ce travail est ci-après décrit :

#### La compréhension du problème métier ou de l’entreprise

La première étape du processus CRISP-DM consiste à comprendre la problématique que le data mining vise à résoudre, à comprendre les objectifs de l’entreprise en matière de data mining. Cela nécessite dans un premier temps de recueillir les besoins de l’entreprise, définir l’objectif principal de l’étude et le traduire en des objectifs spécifiques de data mining comme il a été fait dans le [cadre général](#cadregén). La phase de compréhension du problème comprend également le recensement des ressources disponibles dans l’entreprise et la définition de certains critères essentiels que sont la variable d’intérêt, la population cible, la période à étudier. S’agissant d’une problématique de credit scoring, définir la variable d’intérêt revient à définir le défaut de remboursement.

#### La compréhension et la préparation des données

À la suite de la compréhension du métier, viennent la compréhension et la préparation des données. La finalité de ces deux phases est la constitution d’une base de données structurée, apurée et prête pour la modélisation. Bien que le schéma classique du processus CRISP-DM place la compréhension des données avant la préparation des données, ces deux phases sont prises ensemble dans cette étude car des allers-retours entre l’une et l’autre phase seront nécessaires pour la mener à bien. Ceci sera mieux expliqué dans la présentation de ces deux phases ci-après.

* La **phase de compréhension des données**, comme son nom l’indique, a pour but de s’imprégner des données disponibles pour le data mining. Elle inclut les tâches suivantes :
    * la ***collecte des données*** recensées à l’étape précédente ainsi que les informations nécessaires pour les comprendre (sources et significations) ;
    * la ***description des données*** : elle consiste à relever les caractéristiques relatives à la quantité et à la qualité des données ;
    * la ***vérification de la qualité des données*** : cela consiste en l’identification des données manquantes, des incohérences et des erreurs dans les données collectées ;
    * l’***exploration des données*** : elle permet, grâce à diverses techniques d’analyse des données, de se familiariser avec les données de l’étude. Elle sera faite en trois étapes principales :
        * L’***analyse descriptive unidimensionnelle*** : en observant la distribution des variables, l’analyse univariée donne une vue synthétique des caractéristiques des emprunteurs et de leur comportement en matière de défaillance de remboursement. S’agissant des ***variables qualitatives***, les distributions des différentes modalités seront évaluées par des tris à plat et représentées graphiquement à l’aide de diagrammes barplots. Cela permettra d’identifier les variables à modalités trop nombreuses ou les modalités de fréquence faible qu’il faudra regrouper pour éviter l’influence qu’elles sont susceptibles d’avoir sur les résultats de prochaines analyses. Quant aux ***variables quantitatives***, des résumés numériques tels que le minimum, le maximum, la moyenne, la médiane, les quantiles et l’écart-type seront calculés. Aussi, leur distribution sera représentée graphiquement à l’aide des courbes de densité et des boxplots. Cette représentation permettra d’une part de juger de leur asymétrie, en fonction de laquelle des transformations de type Box-Cox (logarithme, racine carrée, etc.) seront appliquées pour approcher une distribution normale (les modèles de data mining gagnent en précision avec la normalité des variables explicatives) ; et d’autre part d’identifier les valeurs extrêmes qui seront traitées car les modèles tels que la régression logistique sont sensibles à de telles valeurs. Soutenant l’idée selon laquelle les valeurs extrêmes ne sont pas nécessairement aberrantes et leur suppression constituerait une perte d’information, la technique retenue pour leur traitement est la winsorisation. Elle consiste à fixer une limite sur des quantiles (en général le premier et le 99ème centiles) et à ramener à ces valeurs celles qui se situent au-delà.
        * L’***analyse descriptive bidimensionnelle*** : à la suite de l’observation individuelle des attributs, il est important d’une part de croiser chaque variable explicative avec la variable cible, et d’autre part de croiser les variables explicatives entre elles, dans le but d’évaluer les liens existants entre celles-ci. En effet, lors de la construction des modèles de credit scoring, il est important que les variables explicatives intégrées soient, d’une part, fortement liées à la variable d’intérêt et, d’autre part, indépendantes les unes des autres pour éviter les problèmes liés au [sur-apprentissage](https://mrmint.fr/overfitting-et-underfitting-quand-vos-algorithmes-de-machine-learning-derapent "Il y’a sur-apprentissage d’un modèle lorsque celui-ci s’adapte presque parfaitement à l’ensemble de données sur lequel il a été construit, mais ne saurait être généralisé à d’autres ensembles. Dans cette situation, le modèle perd son pouvoir de prédiction sur d’autres échantillons.").
        * L’***analyse descriptive multidimensionnelle*** : disposant d’un jeu de données multivarié (ou multidimensionnel) c’est-à-dire contenant plus de deux variables décrivant plus de deux emprunteurs, une observation plus globale de toutes les variables ensemble est nécessaire pour mieux identifier les relations entre les variables de l’étude et capter les structures de groupe qui pourraient se peaufiner dans le jeu de données. L’analyse exploratoire multidimensionnelle intervient donc en complément de l’exploration bivariée pour mieux cerner les relations entre l’ensemble des variables. Étant majoritairement en présence de variables explicatives quaalitatives, l’analyse des correspondances multiples (ACM). Pour cette étude, il sera question d’identifier la représentation des caractéristiques qui discriminent au mieux les deux groupes d’emprunteurs : ceux à faible risque qui ne font pas défaut de remboursement, et ceux à haut risque qui font défaut de remboursement. Cela permettra d’identifier, parmi les variables explicatives, les plus discriminantes afin d’affiner l’élaboration des modèles.
* La phase de ***préparation des données*** est la plus fastidieuse du data mining, et nécessite d’autant plus de temps et d’énergie que le jeu de données est volumineux. Elle comprend les tâches suivantes :
    * le ***nettoyage des données*** : il s’agit de la gestion des valeurs manquantes (suppression ou remplacement), incohérences et erreurs identifiées ;
    * la ***construction de nouvelles données*** : en général, il est nécessaire de calculer de nouveaux attributs à partir des attributs bruts collectés par l’entreprise, mais également de transformer certains attributs avant de les intégrer dans les modèles ;
        * la ***sélection des données*** : il s’agit du tri des données pertinentes en vue de la modélisation ;
        * le ***fractionnement de la base de données en sous-ensembles d’apprentissage et de test*** : il est recommandé, lorsque la taille de l’ensemble de données disponible pour l’étude le permet, de séparer l’échantillon mère en deux sous-ensembles que sont l’échantillon d’apprentissage et l’échantillon test. L’échantillon d’apprentissage est celui sur lequel les modèles sont construits, et l’échantillon test permet de vérifier l’efficacité des modèles obtenus en apprentissage et sélectionner le meilleur en évitant le biais d’optimisme qui aurait existé si la vérification était faite sur le même échantillon qui a servi à construire le modèle.
        
Ainsi, il sera question de collecter les données, de relever les caractéristiques liées à la quantité et à l’état de ces données, de vérifier leur qualité, de les nettoyer, de les transformer et de construire de nouveaux attributs si nécessaire, d’effectuer une analyse exploratoire sur les données ainsi apurées puis sélectionner les attributs pertinents pour la modélisation et constituer les échantillons d’apprentissage et de test.


#### La modélisation du risque de défaut

La phase de modélisation est considérée comme le coeur du data mining. C’est à ce niveau que les efforts consentis au cours de la compréhension et la préparation des données commencent à être récompensés. Au cours de cette phase, les problématiques de data mining nécessitent le plus souvent de sélectionner plusieurs techniques de modélisation qui seront implémentées avec différents paramètres car le modèle qui résout le mieux le problème de l’étude n’est pas connu a priori.

* **Algorithmes de modélisation**

Avant de construire les modèles, il faut élaborer la façon dont leurs résultats seront testés. Cela nécessite le choix des critères de qualité d’ajustement des modèles et la définition des données sur lesquelles ces critères seront testés. En ce qui concerne la dernière tâche, il s’agit généralement de l’échantillon de test issu du partitionnement de la base entrepris lors de la préparation des données. En somme, la phase de modélisation comprend le choix des techniques à implémenter, le choix des critères de qualité d’ajustement des modèles, la construction des modèles, l’évaluation des modèles et le choix du meilleur modèle.

Pour résoudre la problématique de notre étude, nous souhaitons construire un modèle de prédiction qui attribue à chaque potentiel emprunteur, à partir de ses caractéristiques, un score interprétable comme la probabilité que ce dernier n’honore pas son engagement pour un prêt. Ce score représente le risque de défaillance de l’emprunteur. Pour construire ce modèle, les études précédemment présentées **(San Pedro et al. (2015), Ruiz et al. (2019), Björkegren Grissen (2019), Ots et al. (2020), Shema (2019))** recommandent trois techniques de modélisation, dont deux sont retenues dans ce travail : la régression logistique et l’arbre de décision. La forêt aléatoire ne sera pas appliquée car la population d’étude a une taille suffisamment grande pour se contenter de l’arbre de décision.

##### Régression logistique binaire
    
La régression logistique est une technique de régression qui permet d’exprimer une variable réponse catégorielle (indiquant la survenue d’un évènement donné) sous forme d’un risque (ou probabilité) en fonction des valeurs prises par un ensemble de variables prédictives ou explicatives
pouvant être quantitatives comme qualitatives. Pour cela, elle est très utilisée pour prédire le risque ou la probabilité qu’un évènement se produise. La variable réponse peut être dichotomique (ou binaire, on parle alors de régression logistique binaire), multinomiale ordonnée (régression logistique ordinale) ou multinomiale non ordonnée (régression logistique multinomiale). Dans le cadre de cette étude, la variable réponse est dichotomique, indiquant si un emprunteur donné a
fait défaut de remboursement ou pas.

En plus du fait d’exprimer des relations non linéaires et de modéliser directement la probabilité de survenue d’un évènement, la régression logistique présente l’avantage de produire des coefficients qui correspondent au logarithme des odds ratio (OR) ou rapports des cotes, indiquant la force et le sens de l’association entre les variables explicatives et la variable d’intérêt et faciles à interpréter. L’odds ratio (OR) de chaque variable (ou modalité pour les variables qualitatives) sera obtenu en calculant l’exponentielle du coefficient correspondant. De plus, cette technique offre la possibilité de prendre en compte les interactions entre les variables explicatives. Cependant, sa sensibilité aux individus hors norme (valeurs extrêmes) et l’exigence de l’indépendance des variables explicatives peuvent limiter son utilisation.

##### Arbre de décision
    
Les arbres de décision constituent l’une des principales techniques utilisées en data mining et également en credit scoring, pour la principale raison qu’en plus d’être performants, ils se prêtent
volontiers à une représentation graphique sous une forme arborescente plus simple à interpréter que des résultats purement numériques. Ils permettent de prédire l’affectation d’individus à des classes d’une variable d’intérêt catégorielle à partir de leurs valeurs sur une ou plusieurs variables prédictives (pouvant être quantitatives comme qualitatives) : on parle d’arbres de classification.

La construction d’un arbre de décision consiste en la définition récursive de noeuds et de sousarbres. La définition d’un noeud consiste à choisir conjointement une variable parmi les variables prédictives et une division qui induit une partition en deux classes (sous-arbres) lorsque la variable d’intérêt a deux modalités. Le choix d’une division consiste lui-même à définir une valeur seuil de la variable sélectionnée si elle est quantitative ou à partager en deux groupes ses modalités si elle est qualitative. Chaque noeud correspond ainsi à un sous-échantillon de l’échantillon mère, le noeud initial (racine) correspondant à l’ensemble de l’échantillon.

Trois points doivent essentiellement être traités pour construire un arbre de décision : la sélection de la meilleure division parmi toutes les divisions admissibles (c’est-à-dire produisant des noeuds tous non vides), la décision qu’un noeud est terminal (un noeud terminal est une feuille) et l’affectation de chaque feuille à une des classes de la variable d’intérêt. Selon le critère de division, la règle d’arrêt et d’affectation, il existe plusieurs algorithmes de construction d’arbres de décision dont les plus populaires sont : CHAID (Chi-Square Automation Interaction Detection), C5.0 (de J.R. Quinlan) et CART (Classification And Regression Tree).

Pour cette étude, l’algorithme CART de Breimer et al. (1984) sera implémenté. Cette méthode se base sur l’indice de Gini comme mesure d’impureté et critère de sélection. Précisément, la meilleure division correspond à celle qui maximise la perte d’impureté. En ce qui concerne le critère d’arrêt, l’algorithme CART a la particularité d’intégrer une procédure entièrement automatique comme règle d’arrêt, contrairement aux autres méthodes faisant intervenir une règle dépendante d’un seuil arbitraire. Le critère d’arrêt consiste à ne pas découper un noeud pur, donc un noeud devient terminal lorsqu’il est jugé sans impureté. Pour éviter un découpage trop fin et une instabilité trop grande, un nombre minimal d’individus est souvent fixé pour chaque division d’un noeud. Toutes les partitions de CART sont binaires.

Parlant de leurs propriétés, les arbres de décision performent bien face à des données hétérogènes, manquantes, des variables liées entre elles et des relations non linéaires entre les variables. Aussi, il est recommandé d’user de cette technique sur de grands échantillons pour que les feuilles ne soient pas de densité trop faible. Ceci dit, l’un des défauts majeurs des arbres de décisions est leur instabilité. En effet, des changements sur les noeuds proches de la racine peuvent complètement modifier l’arbre. En solution à ces défauts et pour améliorer la qualité ou la fiabilité des arbres de décision, des techniques telles que le bootstrap, les forêts aléatoires (encore appelées forêts d’arbres décisionnels) ou encore le boosting ont été conçues.

* **Critères de sélection du modèle optimal**

Cette étude voudrait prédire le mieux possible la classe (« mauvais » ou « bon ») des emprunteurs. Sa capacité à le faire est mesurée par le taux de bon classement (TBC) ou le taux d’erreur (TE), premier critère retenu pour évaluer les modèles. Aussi, nous choisissons de rechercher la meilleure sensibilité possible, dans l'optique de permettre à Kiva de limiter au maximum les défauts et donc n'approuver que les prêts des emprunteurs les plus dignes de confiance, et l’aptitude du modèle à détecter les « bons » emprunteurs est mesurée par la sensibilité. Les modèles seront donc sujets à un arbitrage dans le but de trouver le meilleur compromis entre TBC et sensibilité. Par ailleurs, les performances des modèles en test sont, d’une certaine façon, plus indicatrices de leur qualité que celles en apprentissage puisqu’elles
expriment leur pouvoir de généralisation. Le choix du modèle optimal intégrera donc le pouvoir de généralisation.

Les mesures de qualité d’ajustement des modèles suivantes sont alors choisies pour évaluer les modèles : le taux de bon classement (TBC) ou le taux d’erreur (TE), et la sensibilité obtenus à partir de la matrice de confusion, l’aire sous la courbe ROC et la courbe de lift. À ces indicateurs, s’ajoute la facilité d’interprétation qui sera déterminante dans le choix du modèle final.


#### L’évaluation

La précédente évaluation des modèles concernait leur performance statistique, pour vérifier qu’ils sont performants et efficaces en fonction des critères de data mining choisis. Arrivés à ce stade, la plus grande partie de l’étude de data mining a été réalisée. La présente phase est un approfondissement de l’évaluation. Elle consiste à évaluer l’adéquation du modèle aux objectifs de l’entreprise fixés en amont et à vérifier s’il y’a des raisons professionnelles qui rendent le modèle défaillant. À ce niveau, l’on décide s’il faut passer à la phase de déploiement.

#### Le déploiement

La phase de déploiement correspond à l’opérationnalisation du modèle construit. Elle représente l’aboutissement de toute étude de data mining car sans elle, tous les efforts consentis depuis le début de l’étude sont vains. De façon pratique, le déploiement consiste à utiliser les connaissances obtenues à l’aide du data mining pour apporter des améliorations au sein de l’entreprise. Au cours de cette phase, une présentation simple des résultats de l’étude sera faite pour permettre à Kiva de se les approprier.

*Ayant ainsi déroulé les différentes étapes du travail entrepris, l’on amorce leur réalisation par l'importation dynamique et statique de la base de données.*